**goal is to trim 15 510 geographical articles, to charaterize each headphrase by the Geographical Entities (GE) mentionned in the article body**

In [1]:
import sys
sys.path.append('/home/antoine/Documents/GitHub/linkencyclo')
datapath = '/home/antoine/Documents/GitHub/datas/'

from EncycloObject import Article, Book
import pickle

We load a `Book` instance :

In [2]:
with open(datapath+'geobook_22082024.pkl', 'rb') as f:
    mybook = pickle.load(f)
print(mybook)

Book with 15510 articles
plain version of the geo-articles, with the following attributes
['volume', 'numero', 'headword', 'authors', 'text', 'hash', 'artfl', 'gold_coords', 'enccre', 'article_id', 'gold_qid']


In [7]:
with open(datapath+'geobook_plain_22082024.pkl', 'rb') as f:
    plain_geobook = pickle.load(f)
print(plain_geobook)

Book with 15510


In [5]:
# for article in mybook:
#     head = article.headword
#     article.headphrase = head
#     del article.headword

In [9]:
with open(datapath+'geobook_22082024.pkl', 'wb') as f:
    pickle.dump(mybook, f)
    

Example of the Article instance with hash `7/2925/GRENOBLE`

In [3]:
grenoble = mybook._reach_article(headphrase='grenoble')
grenoble.artfl, grenoble.hash, grenoble.text

('https://artflsrv04.uchicago.edu/philologic4.7/encyclopedie0922/navigate/7/2925',
 '7/2925/GRENOBLE',
 'GRENOBLE, Gratianopolis, (Géogr.) ancienne ville de France, capitale du Dauphiné, avec un évêché suffragant de Vienne, & un parlement érigé en 1493 par Louis XI. qui n\'étoit encore que dauphin ;  mais son pere ratifia cette érection deux ans après.\nGrenoble est sur l\'Isere, à onze lieues S O. de Chambéri, quarante-deux N. O. de Turin, seize S. E. de Vienne, cent vingt-quatre S. O. de Paris. Long. suivant Harris, 23d. 31\'. 15". suivant Cassini, 23d. 14\'. 15". latit 45d. 11\'.\nCette ville reçut le nom de Gratianopolis de l\'empereur  Gratien fils de Valentinien I. car elle s\'appelloit  auparavant Cularo ; & c\'est sous ce nom qu\'il en est parlé dans une lettre de Plancus à Cicéron, epist. xxiij. Long-tems après, les Romains l\'érigerent en cité: dans le cinquieme siecle, elle fut assujettie par les Bourguignons, & dans le sixieme par les François Mérovingiens ; ensuite elle a 

In [4]:
grenoble.__dict__.keys()

dict_keys(['volume', 'numero', 'authors', 'text', 'hash', 'artfl', 'gold_coords', 'enccre', 'article_id', 'gold_qid', 'parsed', 'ner', 'headphrase'])

# Part-of-Speech parsing
Stanza doc are stored in the `.parsed` attribute

In [25]:
import stanza
nlpstanza = stanza.Pipeline(lang='fr', processors='tokenize,mwt, pos')

2024-08-22 10:58:49 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-08-22 10:58:49 INFO: Downloaded file to /home/antoine/stanza_resources/resources.json
2024-08-22 10:58:49 INFO: Loading these models for language: fr (French):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2024-08-22 10:58:49 INFO: Using device: cpu
2024-08-22 10:58:49 INFO: Loading: tokenize
2024-08-22 10:58:50 INFO: Loading: mwt
2024-08-22 10:58:50 INFO: Loading: pos
2024-08-22 10:58:50 INFO: Done loading processors!


In [29]:
for article in plain_geobook:
    article.parsed = nlpstanza(article.text)
    #break

# Named Entity Recognition
We apply the BERT model to identify the GE mentionned in each article-body.

From
> 7/2925/GRENOBLE 
> 
> 'GRENOBLE, Gratianopolis, (Géogr.) ancienne ville de France, capitale du Dauphiné, avec un évêché suffragant de Vienne, ...'

We want to store in the `article.ner` attribute the NER tags like :
```python
[
    {'entity_group': 'NC_Spatial', 'score': 0.9601506, 'word': 'ancienne ville', 'start': 26, 'end': 40},
    {'entity_group': 'NP_Spatial', 'score': 0.96639144, 'word': 'France', 'start': 44, 'end': 50},
    {'entity_group': 'NC_Spatial', 'score': 0.96316826, 'word': 'capitale', 'start': 52, 'end': 60},
    {'entity_group': 'NP_Spatial', 'score': 0.9662918, 'word': 'Dauphiné', 'start': 64, 'end': 72},
    ...
]
```


In [30]:
from transformers import pipeline
bert = pipeline("token-classification", model="GEODE/bert-base-french-cased-edda-ner",
                aggregation_strategy="simple")

/home/antoine/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/antoine/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
for article in plain_geobook:
    article.ner = article._apply_pipeline(bert)
    #break

We then add NER tags to the Stanza doc thanks to the method `Article._enrich_stanzadoc`
- native `Token` instances of the Stanza doc receive the NER tags
- native `Span` instances of the Stanza doc receive the contiguous merged NER tags

The GE phrases are normalized :
- `ancienne ville` to `ville`
- `royaume de France` to `france`
- `St. Etienne` to `saint etienne`


In [5]:
# loading stopwords from Solr
with open('solr_stopwords.txt', 'r') as f:
    stopwords = f.read().split('\n') 

In [15]:
for article in mybook:
    article._enrich_stanzadoc(stopwords=stopwords)
    try :
        ncs, nps = article._get_spatial_info(stopwords=stopwords)
    except:
        print(article.headphrase)
        e = mybook._reach_article(headphrase=article.headphrase)
        break
    #break

ABROLHOS


In [34]:
import re
from unidecode import unidecode
from stanza.models.common.doc import Span, Token

VOC_SAINT = f"sainte|saint|sant|san|st|s"
def is_saint(phrase):
    """
    Returns True if the phrase contains a token in the VOC_SAINT list
    """
    return bool(re.search(rf'\b({VOC_SAINT})\b', phrase))


def normalize_span(span, pos=['NOUN', 'PROPN', 'ADJ'], stop_words=None):
    """
    Input :
    - span : Stanza Span instances (stanza.models.common.doc.Span)
    
    Normalizes Proper Nouns Phrases or Common Nouns Phrases by applying the following rules:

    - Proper Nouns Phrase : set pos=[NOUN, 'PROPN', 'ADJ'] to remove tokens diffrent [NOUN, 'PROPN', 'ADJ']
    - Common Nouns Phrase : set pos=[NOUN, 'PROPN'] to remove tokens diffrent [NOUN, 'PROPN']
        - for CNP we also add 'PROPN' because some nouns ('Bourg', 'Isle', 'Ville') are tagged as 'PROPN'
        because they are capitalized
        - other issue : for 'isle' we suspect a out-of-vocabulary issue :
        {"text": "isle", "upos": "X",}
    - detect if the phrase is a saint name and prepend 'saint' in that case        
    - apply lower(), unidecode() to each token
    - remove tokens in a list of stopwords to double-check
    - remove symbols and digits

    Output :
    list of Stanza Span instances with new attribute .norm_text

    """

    norm_text = []

    # if pattern for VOC_SAINT is found, we prepend 'saint'
    if is_saint(unidecode(span.text.lower())):
        norm_text = ['saint']
    
    # keeping only tokens with the right POS
    norm_text.extend(
        [
            unidecode(word.text.lower()) for word in span.words \
                if word.upos in pos \
                    and not is_saint(unidecode(word.text.lower()))                            
        ]
    )

    # remove stopwords
    norm_text = [token for token in norm_text if token not in stop_words]

    # remove symbols and digits
    norm_text = [re.sub(r'[^a-z\s]', '', token) for token in norm_text]

    norm_text = ' '.join(norm_text)
    
    # adding the new attribute
    span.norm_text = norm_text
    #norm_nps.append(np)
    if not hasattr(span, 'norm_text'):
        print(span.text)
    return span

# def normalize_NC(ncs, NC_pos=['NOUN', 'PROPN'], stop_words=None):
#     """
#     Input :
#     - ncs : list of Stanza Span instances (stanza.models.common.doc.Span)
    
#     Normalizes **Noms Communs** Phrases by applying the following rules:
#     - remove tokens not tagged as [NOUN, 'PROPN'] = NC_pos.
#         we also add 'PROPN' because some nouns ('Bourg', 'Isle', 'Ville') are tagged as 'PROPN'
#         other issue : for 'isle' we suspect a out-of-vocabulary issue :
#             # {"text": "isle", "upos": "X",}
#     - apply lower(), unidecode() to each token
#     - remove tokens in a list of stopwords to ensure
#     - remove symbols and digits

#     Output :
#     list of Stanza Span instances with attribute .norm_text
#     """
#     for np in ncs:
#         norm_nps = []

#         # if pattern for VOC_SAINT is found, we delete it and prepend 'saint'
#         if is_saint(unidecode(np.text.lower())):
#             norm_nps = ['saint']
#             continue
        
#         norm_nps.extend(
#             [
#                 unidecode(word.text.lower()) for word in np.words \
#                     if word.upos in NP_pos \
#                         and not is_saint(unidecode(word.text.lower()))                            
#             ]
#         )
#         norm_nps = [token for token in norm_nps if token not in stop_words]

#         norm_nps = [re.sub(r'[^a-z\s]', '', token) for token in norm_nps]
#         norm_nps = ' '.join(norm_nps)
#         np.norm_text = norm_nps
#         #norm_nps.append(np)
#     return nps


In [37]:
nps = [entity_span for entity_span in e.parsed.entities if entity_span.type == 'NP_Spatial']
nps = [normalize_span(span, pos=['NOUN', 'PROPN', 'ADJ'], stop_words=stopwords) for span in nps]

In [40]:
for np in nps:
    print(np.text, np.norm_text)

Sainte-Barbe saint
Brésil bresil


In [41]:
for span in e.nps:
    print(span.text, span.norm_text)

AttributeError: 'Span' object has no attribute 'norm_text'

# Absolute count of NP-Spatial
(Geographic Entities that are NP with a proper Noun)